# Compare brain regions with a custom parcellation

It is often useful to determine spatial overlap or correlations between maps from different brain parcellations, or between a region of interest and a brain map. This tutorial shows a simple case, where a custom map of the Destrieux atlas in MNI152 space is assessed against a group of probabilistic region maps from the Julich-Brain. 

In [ ]:
import siibra
from nilearn import plotting
import numpy as np
import re
atlas = siibra.atlases['human']

## Create a custom Destrieux parcellation from region names and a labelled nifti volume

The Destrieux atlas is ported in Freesurfer and it is provided in the package for standard surfaces. To obtain a decent volumetric form, a transformation from fsaverage to MNI152 was performed using the registration fusion technique by Wu et al. (https://pubmed.ncbi.nlm.nih.gov/29770530/). 

The volumetric map was compute and kindly provided by C. Paquola (Forschungszentrum Julich) and hosted on a public owncloud share (https://fz-juelich.sciebo.de/s/6KwoaksYyhU3br6). Note that the volumetric map is not perfect, it contains some discontinuities. 

For the numbering of parcels, the FreesurferColorLUT.txt which is shipped with [freesurfer](https://surfer.nmr.mgh.harvard.edu/) can be used. The list for Destrieux regions starts at Line 1234, with their label indices having an offset of 10100 (eg: 1038 in the volume is given as 11138 in the lookup table). For convenience, a copy of this file is also available on the above shared folder. 

The first step is to retrieve the labelled volume and region metadata files from the owncloud share. `siibra` has a convenient helper class for grabbing files from this type of repository:

In [ ]:
share = siibra.retrieval.OwncloudConnector(
    server = "https://fz-juelich.sciebo.de",
    share = "6KwoaksYyhU3br6"
)
labelvolume = share.get("aparc.a2009s.mni152.nii")
regionspecs = share.get("FreeSurferColorLUT.txt")

We extract labelindex/regionname mappings from the text file, and use them to create a new `Parcellation` object for the Destrieux atlas.

In [ ]:
# extract the regionname and labelindex from text to define regions
lines = regionspecs.split("\n")
regiondefs = [
    {
        'labelIndex': int(l.split()[0])-10100,
        'name': l.split()[1]
    }
    for l in lines[1233:] 
    if re.match(f'\d.*$',l)
]

# Build parcellation object
destrieux = siibra.core.Parcellation(
    identifier="destrieux",
    name="Volumetric Destrieux parcellation",
    regiondefs = regiondefs,
    maps = [
        siibra.volumes.LocalNiftiVolume(
            name = "Destrieux parcellation in MNI152",
            img = labelvolume,
            space = atlas.get_space('mni152'))]
)

## Computing overlaps of specific probabilistic maps with a parcellation map

We want to compute correlations of specific regions of Julich-Brain with another parcellations. 

`siibra`'s `ParcellationMap` class provides the method `assign` to compute overlaps with a custom map, and returns a list of assignments that is sorted by correlation strength. Each assignment is a 3-tuple, containing the assigned region object, a reference to the used region mask, and a dictionary of overlap scores, which includes amongst others Pearson's correlation coefficient. Note that the first call to `assign` is quite slow, since it will build all the required regional masks. Subsequent calls are fast, since `siibra` caches the mask definitions. 

We are interested to match an arbitrary subset of Julich-Brain regions individually, as well as their combination into a "group map". This can be achieved by creating a custom group region from a regular expression search across region names, using `siibra`'s `find_regions` method.

We put the extraction of the group region and matching to anther parcellation into a function so we can reuse it for different choices of regions and parcellations.



In [ ]:
def assign(jubrain_regex, other_map, min_correlation = 0.3):
    """
    Assign Julich-brain regions matched by the given regex 
    to another parcellation map.
    
    Parameters
    ----------
    
    min_correlation: float
        only matches with a Pearson correlation stronger 
        than this value are considered.
    
    Return
    ------
        Dictionary of assignments per Julich-brain region. 
    """
    import re 
    assignments = {}

    # build the group region in this hemisphere.
    # It will include the group as a parent region, 
    # as well as the individual child regions.
    groupregion, = atlas.find_regions(
        re.compile(jubrain_regex), 
        build_groups=True, 
        groupname = "Group "+jubrain_regex)

    # compute assignments across the group
    for region in groupregion: #
        
        mask = region.build_mask("mni152",maptype='continuous')        
        result = other_map.assign(
            mask, msg=f"Assigning {region.name:20.20}")
        significant = [
            r for r in result 
            if abs(r[2]['correlation']) > min_correlation ]
        if len(significant)>0:
            assignments[region] = significant
                
    return assignments
                

We also define a function for plotting resulting assignments.

In [ ]:
def plot_assignments(assignments):
    
    for query_region, matches in assignments.items():
        for (matched_region, matched_map, scores) in matches:
            view = plotting.plot_stat_map(
                query_region.build_mask("mni152", maptype='continuous'),
                title=f"{matched_region.name} / {query_region.name} "
                    f"(ccoeff={scores['correlation']:.2f})")
            view.add_contours(matched_map)

### Match Destrieux

In [ ]:
destrieux_map = destrieux.get_map(space="mni152")

In [ ]:
assign_destrieux_l = assign('Fo[4-7].*left', destrieux_map)
plot_assignments(assign_destrieux_l)

In [ ]:
assign_destrieux_r = assign('Fo[4-7].*right', destrieux_map)
plot_assignments(assign_destrieux_r)

### Match Desikan/Kiliany

Instead of the custom Destrieux parcels, we perform the same analysis with the predefined DK parcellation shipped with `siibra`.

In [ ]:
dk_map = atlas.get_map('mni152','desikan')

In [ ]:
assignments_dk_l = assign('Fo[4-7].*left', dk_map)
plot_assignments(assignments_dk_l)

In [ ]:
assignments_dk_r = assign('Fo[4-7].*right', dk_map)
plot_assignments(assignments_dk_r)